# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Molecule, Planet, OCEAN_MOLES, SystemConstraint
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, LibourelN2, MolarMasses
import numpy as np
import csv

12:03:23 - atmodeller           - INFO      - atmodeller version 0.1.0


In [2]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='N2', solubility=LibourelN2(), solid_melt_distribution_coefficient=0))
molecules

12:03:23 - atmodeller.core      - INFO      - Creating a molecule: H2O
12:03:23 - atmodeller.core      - INFO      - Creating a molecule: H2
12:03:23 - atmodeller.core      - INFO      - Creating a molecule: O2
12:03:23 - atmodeller.core      - INFO      - Creating a molecule: CO
12:03:23 - atmodeller.core      - INFO      - Creating a molecule: CO2
12:03:23 - atmodeller.core      - INFO      - Creating a molecule: N2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x1265635d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x115308ad0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x1152fa190>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x125d55690>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, for

In [3]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, planet=planet)

12:03:23 - atmodeller.core      - INFO      - Creating a new planet
12:03:23 - atmodeller.core      - INFO      - Mantle mass (kg) = 4.208261222595111e+24
12:03:23 - atmodeller.core      - INFO      - Mantle melt fraction = 1.0
12:03:23 - atmodeller.core      - INFO      - Core mass fraction = 0.295334691460966
12:03:23 - atmodeller.core      - INFO      - Planetary radius (m) = 6371000.0
12:03:23 - atmodeller.core      - INFO      - Planetary mass (kg) = 5.972e+24
12:03:23 - atmodeller.core      - INFO      - Surface temperature (K) = 2000.000000
12:03:23 - atmodeller.core      - INFO      - Surface gravity (m/s^2) = 9.819973426224687
12:03:23 - atmodeller.core      - INFO      - Oxygen fugacity model (mantle) = IronWustiteBufferOneill
12:03:23 - atmodeller.core      - INFO      - Oxygen fugacity log10 shift = 0.000000
12:03:23 - atmodeller.core      - INFO      - Creating a new interior-atmosphere system
12:03:23 - atmodeller.core      - INFO      - Molecules = ['CO', 'H2', 'N2', 'O2

In [4]:
number_of_realisations: int = 2

# Parameters are normally distributed between bounds.
number_ocean_moles: np.ndarray = np.random.uniform(1, 10, number_of_realisations)
ch_ratios: np.ndarray = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts: np.ndarray = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out: list[dict[str, float]] = []

# ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
N_ppmw: float = 2.8

# The nitrogen mass is constant.
mass_N: float = N_ppmw * 1.0e-6 * planet.mantle_mass

for realisation in range(number_of_realisations):

    mass_H: float = number_ocean_moles[realisation] * OCEAN_MOLES * MolarMasses().H2
    mass_C: float = ch_ratios[realisation] * mass_H
    constraints: list[SystemConstraint] = [
        SystemConstraint(species="H", value=mass_H, field="mass"),
        SystemConstraint(species="C", value=mass_C, field="mass"),
        SystemConstraint(species="N", value=mass_N, field="mass")
    ]
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    planet.fo2_shift = fo2_shifts[realisation]
    out_realisation = interior_atmosphere.solve(constraints, fo2_constraint=True)

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = planet.fo2_shift

    out.append(out_realisation)

    filename: str = "equilibrium_atmosphere_monte_carlo.csv"
    print("Writing output to: %s", filename)
    fieldnames: list[str] = list(out[0].keys())
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer: csv.DictWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(out)

12:03:23 - atmodeller.core      - INFO      - Constraints: [SystemConstraint(species='H', value=6.312373681146521e+20, field='mass'),
 SystemConstraint(species='C', value=4.2263427944109664e+20, field='mass'),
 SystemConstraint(species='N', value=1.178313142326631e+19, field='mass')]
12:03:23 - atmodeller.core      - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
12:03:23 - atmodeller.core      - INFO      - Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 3.91
12:03:23 - atmodeller.core      - INFO      - 3 additional (not pressure) constraint(s) are necessary to solve the system
12:03:23 - atmodeller.core      - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
12:03:23 - atmodeller.core      - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
12:03:23 - atmodeller.core      - INFO      - Row 02: Setting O2 partial pressure
12:03:23 - atmodeller.core      - INFO      - Adding

Writing output to: %s equilibrium_atmosphere_monte_carlo.csv
Writing output to: %s equilibrium_atmosphere_monte_carlo.csv


The simulation data is output as:

In [5]:
out

[{'CO': 14.643843749066628,
  'H2': 0.1018423916951978,
  'N2': 3.1661955828993826,
  'O2': 0.0003184601738388703,
  'CO2': 200.86656379467922,
  'H2O': 6.2703557769594696,
  'number_ocean_moles': 4.072660772269923,
  'ch_ratio': 0.6695330485636478,
  'fo2_shift': 3.9108789699691604},
 {'CO': 76.66647842308372,
  'H2': 1.2317302311693519,
  'N2': 2.7008654611786764,
  'O2': 2.1439370174052424e-06,
  'CO2': 86.28524297743147,
  'H2O': 6.222395928002399,
  'number_ocean_moles': 4.061054062035693,
  'ch_ratio': 0.5267876822488561,
  'fo2_shift': 1.7390358650386952}]